# Automated ML

Import Dependencies. In the cell below, import all the dependencies.

In [1]:
# from the examples done in the class - eliminated things by trial and error
from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

# needed for the logging
import sklearn

## Dataset

### Overview
The data set the Standford Policing Project - data about traffic stops and the result of those stops. From the Kaggle web site:   https://www.kaggle.com/faressayah/stanford-open-policing-project.

The model will look to predice whether a stop will result in finding drug and thus a drug arrest. 

This cell below will load the data.  

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automlExperiment'

experiment=Experiment(ws, experiment_name)

Make sure the data is correct by printing it out to the screen. 

In [3]:
# create a TabularDataset from a dataset

dataset = Dataset.get_by_name(ws, name='police_project')
police_records = dataset.to_pandas_dataframe()

# preview the first 3 rows of the dataset
police_records.head(10)

,stop_date,stop_time,county_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,NA,1,20,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
1,1/18/2005,8:15,NA,1,40,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
2,1/23/2005,23:15,NA,1,33,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
3,2/20/2005,17:15,NA,1,19,White,Other,FALSE,NA,Arrest Driver,TRUE,16-30 Min,FALSE
4,3/14/2005,10:00,NA,0,21,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
5,3/23/2005,9:45,NA,1,23,Black,Equipment,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
6,4/1/2005,17:30,NA,1,36,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
7,6/6/2005,13:20,NA,0,19,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
8,7/13/2005,10:15,NA,1,35,Black,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE
9,7/13/2005,15:45,NA,1,35,White,Speeding,FALSE,NA,Citation,FALSE,0-15 Min,FALSE


## AutoML Configuration

AutoML using many machine learning methods to determine the best mode. The settings set up AutoML to find the best classification model based on accuracy. 



In [4]:
automl_settings = {
    "task": 'classification',
    "iterations": 30,
    "experiment_timeout_minutes": 15,
    "primary_metric": 'accuracy',
    "training_data": dataset,
    "label_column_name": 'drugs_related_stop',
    "n_cross_validations": 3,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1,
    "debug_log" : "automl_errors.log"
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

## Run Details


Use the `RunDetails` widget to show the different experiments.

In [5]:
#  Submit your experiment
experiment_run = experiment.submit(automl_config)
RunDetails(experiment_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [6]:
experiment_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|815                              |TRUE                             |91741                                 |
+---------------------------------+----------

{'runId': 'AutoML_9cd16b49-bdbb-4146-b318-16de190d5c6f',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T23:09:11.526864Z',
 'endTimeUtc': '2021-01-29T23:26:55.514798Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automlExperiment","subscription_id":"091ee12e-98ac-4172-b48e-fa981b519954","resource_group":"MiyachiGroup","workspace_name":"Udacity","region":"westus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":2,"featurization":"auto","_ignore_packag

In [6]:
RunDetails(experiment_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Retrieve and save your best automl model.
best_automl_run, best_model = experiment_run.get_output()
print(best_model._final_estimator)
print(best_automl_run)
print(best_automl_run.properties['score'])

print(best_model)
print('Best Run Id: ', best_automl_run.id)
#print('\n Accuracy:', best_automl_run.properties['accuracy'])

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('1',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
                                                                              gamma=0,
              

In [8]:
# Print out for project requirements
print(best_automl_run)

Run(Experiment: automlExperiment,
Id: AutoML_9cd16b49-bdbb-4146-b318-16de190d5c6f_20,
Type: None,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

Register the model, create an inference config and deploy the model as a web service.

In [10]:
model = best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')
model.download() 

'outputs'

In [11]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
environment = Environment.from_conda_specification('my-automl-environment', './outputs/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

service = model.deploy(
    workspace=ws,
    name="automl-deploy-1",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [12]:
service.update(enable_app_insights=True)

In the cell below, send a request to the web service you deployed to test it. The field to predict is the last column.  

In [13]:
ROW_ID = 0
x = police_records.iloc[ROW_ID, :-1].to_dict()
y = police_records.iloc[ROW_ID, -1]
print(x)

{'stop_date': '1/2/2005', 'stop_time': '1:55', 'county_name': 'NA', 'driver_gender': '1', 'driver_age': '20', 'driver_race': 'White', 'violation': 'Speeding', 'search_conducted': 'FALSE', 'search_type': 'NA', 'stop_outcome': 'Citation', 'is_arrested': 'FALSE', 'stop_duration': '0-15 Min'}


Convert the data to JSON and then submit it to the web service deployed. 

In [14]:
import json
import numpy as np
input_payload = json.dumps({
    'data': x
})

output = service.run(input_payload)
print(f"Predicted: {output}\nExpected: {y}")

Predicted: ['FALSE']
Expected: FALSE


In [15]:


import requests
import json
scoringuri = service.scoring_uri
#key=primary

data={  "data":
 #{
   #     'stop_date':'1/2/2005','stop_time': '1:55','county_name':'NA','driver_gender':'1','driver_age':'33','driver_race':'White','violation':'Speeding','search_conducted':'FALSE','search_type':'NA','stop_outcome':'Citation','is_arrested' :'FALSE','stop_duration':'0-15 Min'
     #   },
        {'stop_date':'10/1/2005','stop_time':'0:00','county_name':'NA','driver_gender':'1','driver_age':'17','driver_race':'White','violation':'Equipment','search_conducted':'TRUE','search_type':'Probable Cause','stop_outcome':'Arrest Driver','is_arrested' :'TRUE','stop_duration':'16-30 Min'}
}


input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
# headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

["TRUE"]


In the cell below, print the logs of the web service and delete the service

In [64]:
logs = service.get_logs()
print(logs)
service.delete()

/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-24T22:05:59,374435233+00:00 - iot-server/run 
2021-01-24T22:05:59,374747136+00:00 - gunicorn/run 
2021-01-24T22:05:59,376312853+00:00 - nginx/run 
2021-01-24T22:05:59,379499686+00:00 - rsyslog/run 
rsyslogd